In [2]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
pd.options.display.max_colwidth = 75
pd.options.display.max_columns = 20
np.set_printoptions(precision=4, suppress=True)

# Data Loading, Storage, and File Formats

The term parsing is also sometimes used to describe loading text data and interpreting it as tables and different data types. I’m going to focus on data input and output using pandas, though there are numerous tools in other libraries to help with reading and writing data in various formats.

Input and output typically fall into a few main categories: reading text files and other more efficient on-disk formats, loading data from databases, and interacting with network sources like web APIs.

### Reading and Writing Data in Text Format
pandas features a number of functions for reading tabular data as a DataFrame object. Table 6.1 summarizes some of them; pandas.read_csv is one of the most frequently used in this book. We will look at binary data formats later in Binary Data Formats.

![alt text](d1.png "Title")

I’ll give an overview of the mechanics of these functions, which are meant to convert text data into a DataFrame. The optional arguments for these functions may fall into a few categories:

Indexing
Can treat one or more columns as the returned DataFrame, and whether to get column names from the file, arguments you provide, or not at all.

Type inference and data conversion
Includes the user-defined value conversions and custom list of missing value markers.

Date and time parsing
Includes a combining capability, including combining date and time information spread over multiple columns into a single column in the result.

Iterating
Support for iterating over chunks of very large files.

Unclean data issues
Includes skipping rows or a footer, comments, or other minor things like numeric data with thousands separated by commas.

Because of how messy data in the real world can be, some of the data loading functions (especially pandas.read_csv) have accumulated a long list of optional arguments over time. It's normal to feel overwhelmed by the number of different parameters (pandas.read_csv has around 50). The online pandas documentation has many examples about how each of these works, so if you're struggling to read a particular file, there might be a similar enough example to help you find the right parameters.

Some of these functions perform type inference, because the column data types are not part of the data format. That means you don’t necessarily have to specify which columns are numeric, integer, Boolean, or string. Other data formats, like HDF5, ORC, and Parquet, have the data type information embedded in the format.

Handling dates and other custom types can require extra effort.

Let’s start with a small comma-separated values (CSV) text file:

see ex1.csv

Since this is comma-delimited, we can then use pandas.read_csv to read it into a DataFrame:


In [3]:
df = pd.read_csv("examples/ex1.csv")
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


A file will not always have a header row. Consider this file:

see ex2.csv

To read this file, you have a couple of options. You can allow pandas to assign default column names, or you can specify names yourself:

In [4]:
pd.read_csv("examples/ex2.csv", header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
pd.read_csv("examples/ex2.csv", names=["aaa", "b", "c", "d", "message"])

,aaa,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Suppose you wanted the message column to be the index of the returned DataFrame. You can either indicate you want the column at index 4 or named "message" using the index_col argument:

In [8]:
names = ["a", "b", "c", "d", "message"]
pd.read_csv("examples/ex2.csv", names=names, index_col="message")

,a,b,c,d
m,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


If you want to form a hierarchical index (discussed in Ch 8.1: Hierarchical Indexing) from multiple columns, pass a list of column numbers or names:

see csv_mindex.csv

In [9]:
parsed = pd.read_csv("examples/csv_mindex.csv",index_col=["key1", "key2"])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In some cases, a table might not have a fixed delimiter, using whitespace or some other pattern to separate fields. Consider a text file that looks like this:

see ex3.txt

While you could do some munging by hand, the fields here are separated by a variable amount of whitespace. In these cases, you can pass a regular expression as a delimiter for pandas.read_csv. This can be expressed by the regular expression \s+, so we have then:

In [10]:
result = pd.read_csv("examples/ex3.txt", sep="\s+")
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


Because there was one fewer column name than the number of data rows, pandas.read_csv infers that the first column should be the DataFrame’s index in this special case.

The file parsing functions have many additional arguments to help you handle the wide variety of exception file formats that occur (see a partial listing in Table 6.2). For example, you can skip the first, third, and fourth rows of a file with skiprows:

see ex4.csv

In [11]:

pd.read_csv("examples/ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Handling missing values is an important and frequently nuanced part of the file reading process. Missing data is usually either not present (empty string) or marked by some sentinel (placeholder) value. By default, pandas uses a set of commonly occurring sentinels, such as NA and NULL:

see ex5.csv

In [15]:
result = pd.read_csv("examples/ex5.csv")
result

,something,a,b,c,d,message
0,one,1,2,3.0,NaN,NaN
1,two,5,6,NaN,8.0,world
2,three,9,10,11.0,12.0,foo


Recall that pandas outputs missing values as NaN, so we have two null or missing values in result:



In [16]:
pd.isna(result)

,something,a,b,c,d,message
0,False,False,False,False,True,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


The na_values option accepts a sequence of strings to add to the default list of strings recognized as missing:

In [17]:
result = pd.read_csv("examples/ex5.csv", na_values=["foo"])
result

,something,a,b,c,d,message
0,one,1,2,3.0,NaN,NaN
1,two,5,6,NaN,8.0,world
2,three,9,10,11.0,12.0,NaN


pandas.read_csv has a list of many default NA value representations, but these defaults can be disabled with the keep_default_na option:

In [18]:
result2 = pd.read_csv("examples/ex5.csv", keep_default_na=False)
result2

,something,a,b,c,d,message
0,one,1,2,3,NULL,NA
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [19]:
result2.isna()


,something,a,b,c,d,message
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False


In [20]:
result3 = pd.read_csv("examples/ex5.csv", keep_default_na=False, na_values=["foo"])
result3


,something,a,b,c,d,message
0,one,1,2,3,NULL,NA
1,two,5,6,,8,world
2,three,9,10,11,12,NaN


In [21]:
result3.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,True


Different NA sentinels can be specified for each column in a dictionary:

In [22]:
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
pd.read_csv("examples/ex5.csv", na_values=sentinels,keep_default_na=False)

,something,a,b,c,d,message
0,one,1,2,3,NULL,NaN
1,NaN,5,6,,8,world
2,three,9,10,11,12,NaN


Table 6.2 lists some frequently used options in pandas.read_csv.

![alt text](d2.png "Title")
![alt text](d3.png "Title")

## Reading Text Files in Pieces
When processing very large files or figuring out the right set of arguments to correctly process a large file, you may want to read only a small piece of a file or iterate through smaller chunks of the file.


Now we have:

In [23]:
result = pd.read_csv("examples/ex6.csv")
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


The elipsis marks ... indicate that rows in the middle of the DataFrame have been omitted.

If you want to read only a small number of rows (avoiding reading the entire file), specify that with nrows:

In [27]:
pd.read_csv("examples/ex6.csv", skiprows=10, nrows=10)

,-1.74087710829,-1.16041658062,-1.63783027929,2.17220101238,G
0,0.240564,-0.328249,1.252155,1.072796,8
1,0.764018,1.165476,-0.639544,1.495258,R
2,0.571035,-0.310537,0.582437,-0.298765,1
3,2.317658,0.430710,-1.334216,0.199679,P
4,1.547771,-1.119753,-2.277634,0.329586,J
5,-1.310608,0.401719,-1.000987,1.156708,E
6,-0.088496,0.634712,0.153324,0.415335,B
7,-0.018663,-0.247487,-1.446522,0.750938,A
8,-0.070127,-1.579097,0.120892,0.671432,F
9,-0.194678,-0.492039,2.359605,0.319810,H


To read a file in pieces, specify a chunksize as a number of rows:

In [28]:
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)
type(chunker)

pandas.io.parsers.readers.TextFileReader

In [ ]:
chunker

The TextFileReader object returned by pandas.read_csv allows you to iterate over the parts of the file according to the chunksize. For example, we can iterate over ex6.csv, aggregating the value counts in the "key" column, like so:

In [29]:
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)

tot = pd.Series([], dtype='int64')
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)

We have then:

In [30]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

TextFileReader is also equipped with a get_chunk method that enables you to read pieces of an arbitrary size.

## Writing Data to Text Format
Data can also be exported to a delimited format. Let’s consider one of the CSV files read before:

In [31]:
data = pd.read_csv("examples/ex5.csv")
data

,something,a,b,c,d,message
0,one,1,2,3.0,NaN,NaN
1,two,5,6,NaN,8.0,world
2,three,9,10,11.0,12.0,foo


Using DataFrame’s to_csv method, we can write the data out to a comma-separated file:

In [32]:
data.to_csv("examples/out.csv")

Other delimiters can be used, of course (writing to sys.stdout so it prints the text result to the console rather than a file):

In [33]:
data.to_csv("examples/out.csv", sep="|")

Missing values appear as empty strings in the output. You might want to denote them by some other sentinel value:

In [35]:
import sys
data.to_csv("examples/out_new.csv", na_rep="NULL")

With no other options specified, both the row and column labels are written. Both of these can be disabled:

In [36]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,,
two,5,6,,8.0,world
three,9,10,11.0,12.0,foo


You can also write only a subset of the columns, and in an order of your choosing:

In [37]:
data.to_csv(sys.stdout, index=False, columns=["a", "b", "c"])

a,b,c
1,2,3.0
5,6,
9,10,11.0


## Working with Other Delimited Formats
It's possible to load most forms of tabular data from disk using functions like pandas.read_csv. In some cases, however, some manual processing may be necessary. It’s not uncommon to receive a file with one or more malformed lines that trip up pandas.read_csv. To illustrate the basic tools, consider a small CSV file:

see ex7.csv

For any file with a single-character delimiter, you can use Python’s built-in csv module. To use it, pass any open file or file-like object to csv.reader:

In [38]:
import csv
f = open("examples/ex7.csv")
reader = csv.reader(f)

Iterating through the reader like a file yields lists of values with any quote characters removed:



In [39]:
for line in reader:
    print(line)
f.close()

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


From there, it’s up to you to do the wrangling necessary to put the data in the form that you need. Let's take this step by step. First, we read the file into a list of lines:

In [40]:
with open("examples/ex7.csv") as f:
    lines = list(csv.reader(f))
lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]

Then we split the lines into the header line and the data lines:



In [41]:
header, values = lines[0], lines[1:]
header

['a', 'b', 'c']

In [42]:
values

[['1', '2', '3'], ['1', '2', '3']]

Then we can create a dictionary of data columns using a dictionary comprehension and the expression zip(*values) (beware that this will use a lot of memory on large files), which transposes rows to columns:

In [43]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

## JSON Data
JSON (short for JavaScript Object Notation) has become one of the standard formats for sending data by HTTP request between web browsers and other applications. It is a much more free-form data format than a tabular text form like CSV. Here is an example:

In [45]:
obj = """
{"name": "Wes",
 "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},
              {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]
}
"""

JSON is very nearly valid Python code with the exception of its null value null and some other nuances (such as disallowing trailing commas at the end of lists). The basic types are objects (dictionaries), arrays (lists), strings, numbers, Booleans, and nulls. All of the keys in an object must be strings. There are several Python libraries for reading and writing JSON data. I’ll use json here, as it is built into the Python standard library. To convert a JSON string to Python form, use json.loads:

In [46]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'cities_lived': ['Akron', 'Nashville', 'New York', 'San Francisco'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 34, 'hobbies': ['guitars', 'soccer']},
  {'name': 'Katie', 'age': 42, 'hobbies': ['diving', 'art']}]}

In [47]:
type(result)

dict

In [48]:
result['name']

'Wes'

In [49]:
result['cities_lived']

['Akron', 'Nashville', 'New York', 'San Francisco']

In [50]:
result['siblings'][0]['age']

34

json.dumps, on the other hand, converts a Python object back to JSON:

In [51]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"], "pet": null, "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]}, {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]}'

In [52]:
type(asjson)

str

How you convert a JSON object or list of objects to a DataFrame or some other data structure for analysis will be up to you. Conveniently, you can pass a list of dictionaries (which were previously JSON objects) to the DataFrame constructor and select a subset of the data fields:

In [53]:
siblings = pd.DataFrame(result["siblings"], columns=["name", "age"])
siblings

,name,age
0,Scott,34
1,Katie,42


In [54]:
siblings = pd.DataFrame(result)
siblings

ValueError: All arrays must be of the same length

The pandas.read_json can automatically convert JSON datasets in specific arrangements into a Series or DataFrame. For example:

see example.json

The default options for pandas.read_json assume that each object in the JSON array is a row in the table:

In [56]:
data = pd.read_json("examples/example.json")
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


## XML and HTML: Web Scraping
Python has many libraries for reading and writing data in the ubiquitous HTML and XML formats. Examples include lxml, Beautiful Soup, and html5lib. While lxml is comparatively much faster in general, the other libraries can better handle malformed HTML or XML files.

pandas has a built-in function, pandas.read_html, which uses all of these libraries to automatically parse tables out of HTML files as DataFrame objects. To show how this works, I downloaded an HTML file (used in the pandas documentation) from the US FDIC showing bank failures.1 First, you must install some additional libraries used by read_html:

conda install lxml beautifulsoup4 html5lib
If you are not using conda, pip install lxml should also work.

The pandas.read_html function has a number of options, but by default it searches for and attempts to parse all tabular data contained within <table> tags. The result is a list of DataFrame objects:

In [57]:
tables = pd.read_html("examples/fdic_failed_bank_list.html")
tables

[                             Bank Name             City  ST   CERT  \
 0                          Allied Bank         Mulberry  AR     91   
 1         The Woodbury Banking Company         Woodbury  GA  11297   
 2               First CornerStone Bank  King of Prussia  PA  35312   
 3                   Trust Company Bank          Memphis  TN   9956   
 4           North Milwaukee State Bank        Milwaukee  WI  20364   
 ..                                 ...              ...  ..    ...   
 542                 Superior Bank, FSB         Hinsdale  IL  32646   
 543                Malta National Bank            Malta  OH   6629   
 544    First Alliance Bank & Trust Co.       Manchester  NH  34264   
 545  National State Bank of Metropolis       Metropolis  IL   3815   
 546                   Bank of Honolulu         Honolulu  HI  21029   
 
                    Acquiring Institution        Closing Date  \
 0                           Today's Bank  September 23, 2016   
 1              

In [59]:
len(tables)
failures = tables[0]
failures

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"
...,...,...,...,...,...,...,...
542,"Superior Bank, FSB",Hinsdale,IL,32646,"Superior Federal, FSB","July 27, 2001","August 19, 2014"
543,Malta National Bank,Malta,OH,6629,North Valley Bank,"May 3, 2001","November 18, 2002"
544,First Alliance Bank & Trust Co.,Manchester,NH,34264,Southern New Hampshire Bank & Trust,"February 2, 2001","February 18, 2003"
545,National State Bank of Metropolis,Metropolis,IL,3815,Banterra Bank of Marion,"December 14, 2000","March 17, 2005"


Because failures has many columns, pandas inserts a line break character \.

As you will learn in later chapters, from here we could proceed to do some data cleaning and analysis, like computing the number of bank failures by year:

In [60]:
close_timestamps = pd.to_datetime(failures["Closing Date"])
close_timestamps

0     2016-09-23
1     2016-08-19
2     2016-05-06
3     2016-04-29
4     2016-03-11
         ...    
542   2001-07-27
543   2001-05-03
544   2001-02-02
545   2000-12-14
546   2000-10-13
Name: Closing Date, Length: 547, dtype: datetime64[ns]

In [62]:
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2015      8
2016      5
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, dtype: int64

Parsing XML with lxml.objectify
XML is another common structured data format supporting hierarchical, nested data with metadata. 

Earlier, I showed the pandas.read_html function, which uses either lxml or Beautiful Soup under the hood to parse data from HTML. XML and HTML are structurally similar, but XML is more general. Here, I will show an example of how to use lxml to parse data from a more general XML format.

For many years, the New York Metropolitan Transportation Authority (MTA) published a number of data series about its bus and train services in XML format. Here we’ll look at the performance data, which is contained in a set of XML files. Each train or bus service has a different file (like Performance_MNR.xml for the Metro-North Railroad) containing monthly data as a series of XML records that look like this:



<INDICATOR>
  <INDICATOR_SEQ>373889</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>
  <DESCRIPTION>Percent of the time that escalators are operational
  systemwide. The availability rate is based on physical observations performed
  the morning of regular business days only. This is a new indicator the agency
  began reporting in 2009.</DESCRIPTION>
  <PERIOD_YEAR>2011</PERIOD_YEAR>
  <PERIOD_MONTH>12</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.00</YTD_TARGET>
  <YTD_ACTUAL></YTD_ACTUAL>
  <MONTHLY_TARGET>97.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>



Using lxml.objectify, we parse the file and get a reference to the root node of the XML file with getroot:

In [63]:
from lxml import objectify

path = "datasets/mta_perf/Performance_MNR.xml"
with open(path) as f:
    parsed = objectify.parse(f)
root = parsed.getroot()

root.INDICATOR returns a generator yielding each <INDICATOR> XML element. For each record, we can populate a dictionary of tag names (like YTD_ACTUAL) to data values (excluding a few tags) by running the following code:

In [70]:
data = []

skip_fields = ["PARENT_SEQ", "INDICATOR_SEQ",
               "DESIRED_CHANGE", "DECIMAL_PLACES"]

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [65]:
data

[{'AGENCY_NAME': 'Metro-North Railroad',
  'INDICATOR_NAME': 'On-Time Performance (West of Hudson)',
  'DESCRIPTION': 'Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate service on these lines.\n',
  'PERIOD_YEAR': 2008,
  'PERIOD_MONTH': 1,
  'CATEGORY': 'Service Indicators',
  'FREQUENCY': 'M',
  'INDICATOR_UNIT': '%',
  'YTD_TARGET': 95.0,
  'YTD_ACTUAL': 96.9,
  'MONTHLY_TARGET': 95.0,
  'MONTHLY_ACTUAL': 96.9},
 {'AGENCY_NAME': 'Metro-North Railroad',
  'INDICATOR_NAME': 'On-Time Performance (West of Hudson)',
  'DESCRIPTION': 'Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate 

In [ ]:
type(data)

Lastly, convert this list of dictionaries into a DataFrame:



In [66]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8


pandas's pandas.read_xml function turns this process into a one-line expression:

In [67]:
path = "datasets/mta_perf/Performance_MNR.xml"
perf2 = pd.read_xml(path)
perf2.head()

,INDICATOR_SEQ,PARENT_SEQ,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,DESIRED_CHANGE,INDICATOR_UNIT,DECIMAL_PLACES,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,1,Service Indicators,M,U,%,1,95.00,96.90,95.00,96.90
1,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,2,Service Indicators,M,U,%,1,95.00,96.00,95.00,95.00
2,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,3,Service Indicators,M,U,%,1,95.00,96.30,95.00,96.90
3,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,4,Service Indicators,M,U,%,1,95.00,96.80,95.00,98.30
4,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,5,Service Indicators,M,U,%,1,95.00,96.60,95.00,95.80


In [83]:
perf2 = perf2.drop(skip_fields, axis = 1)


KeyError: "['PARENT_SEQ', 'INDICATOR_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACES'] not found in axis"

In [84]:
perf2.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,1,Service Indicators,M,%,95.00,96.90,95.00,96.90
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,2,Service Indicators,M,%,95.00,96.00,95.00,95.00
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,3,Service Indicators,M,%,95.00,96.30,95.00,96.90
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,4,Service Indicators,M,%,95.00,96.80,95.00,98.30
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at their destinations within 5 m...,2008,5,Service Indicators,M,%,95.00,96.60,95.00,95.80


For more complex XML documents, refer to the docstring for pandas.read_xml which describes how to do selections and filters to extract a particular table of interest.

### Binary Data Formats
One simple way to store (or serialize) data in binary format is using Python’s built-in pickle module. pandas objects all have a to_pickle method that writes the data to disk in pickle format:

In [85]:
frame = pd.read_csv("examples/ex1.csv")
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [87]:
frame.to_pickle("examples/frame_pickle")

Pickle files are in general readable only in Python. You can read any "pickled" object stored in a file by using the built-in pickle directly, or even more conveniently using pandas.read_pickle:

In [88]:
pd.read_pickle("examples/frame_pickle")

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


## Reading Microsoft Excel Files
pandas also supports reading tabular data stored in Excel 2003 (and higher) files using either the pandas.ExcelFile class or pandas.read_excel function. Internally, these tools use the add-on packages xlrd and openpyxl to read old-style XLS and newer XLSX files, respectively. These must be installed separately from pandas using pip or conda.

To use pandas.ExcelFile, create an instance by passing a path to an xls or xlsx file:

In [ ]:
xlsx = pd.ExcelFile("examples/ex1.xlsx")

In [ ]:
xlsx

This object can show you the list of available sheet names in the file:



In [ ]:
xlsx.sheet_names

Data stored in a sheet can then be read into DataFrame with parse:



In [ ]:
xlsx.parse(sheet_name="Sheet1")

This Excel table has an index column, so we can indicate that with the index_col argument:



In [ ]:
xlsx.parse(sheet_name="Sheet1", index_col=0)

If you are reading multiple sheets in a file, then it is faster to create the pandas.ExcelFile, but you can also simply pass the filename to pandas.read_excel:

In [ ]:
frame = pd.read_excel("examples/ex1.xlsx", sheet_name="Sheet1", index_col=0)
frame

In [ ]:
frame.iloc[1,1] = np.nan

To write pandas data to Excel format, you must first create an ExcelWriter, then write data to it using the pandas object's to_excel method:

In [ ]:
writer = pd.ExcelWriter("examples/ex2.xlsx")
frame.to_excel(writer, "Sheet1")
writer.close()

You can also pass a file path to to_excel and avoid the ExcelWriter:



In [ ]:
frame.to_excel("examples/ex3.xlsx")

## Using HDF5 Format
HDF5 is a respected file format intended for storing large quantities of scientific array data. It is available as a C library, and it has interfaces available in many other languages, including Java, Julia, MATLAB, and Python. The “HDF” in HDF5 stands for hierarchical data format. Each HDF5 file can store multiple datasets and supporting metadata. Compared with simpler formats, HDF5 supports on-the-fly compression with a variety of compression modes, enabling data with repeated patterns to be stored more efficiently. HDF5 can be a good choice for working with datasets that don't fit into memory, as you can efficiently read and write small sections of much larger arrays.

To get started with HDF5 and pandas, you must first install PyTables by installing the tables package with conda.

While it's possible to directly access HDF5 files using either the PyTables or h5py libraries, pandas provides a high-level interface that simplifies storing Series and DataFrame objects. The HDFStore class works like a dictionary and handles the low-level details:

In [ ]:
frame = pd.DataFrame({"a": np.random.standard_normal(100)})
frame

In [ ]:
store = pd.HDFStore("examples/mydata.h5")
store.put("obj1", frame)
store

Objects contained in the HDF5 file can then be retrieved with the same dictionary-like API:



In [ ]:
store["obj1"]

In [ ]:
tmp = store.select("obj1", where=["index >= 10 and index <= 15"])
tmp


HDFStore supports two storage schemas, "fixed" and "table" (the default is "fixed"). The latter is generally slower, but it supports query operations using a special syntax:

In [ ]:
store.put("obj2", frame, format="table")
tmp = store.select("obj2", where=["index >= 10 and index <= 15"])
tmp


In [ ]:
store.close()

The put is an explicit version of the store["obj2"] = frame method but allows us to set other options like the storage format.

The pandas.read_hdf function gives you a shortcut to these tools:



In [ ]:
frame.to_hdf("examples/mydata.h5", "obj3", format="table")
pd.read_hdf("examples/mydata.h5", "obj3", where=["index < 5"])

If you'd like, you can delete the HDF5 file you created, like so:



In [ ]:
import os
os.remove("examples/mydata.h5")

## Interacting with Web APIs
Many websites have public APIs providing data feeds via JSON or some other format. There are a number of ways to access these APIs from Python; one method that I recommend is the requests package, which can be installed with pip or conda:

conda install requests
To find the last 30 GitHub issues for pandas on GitHub, we can make a GET HTTP request using the add-on requests library:

In [ ]:
import requests
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
resp = requests.get(url)
resp.raise_for_status()
resp

It's a good practice to always call raise_for_status after using requests.get to check for HTTP errors.

The response object’s json method will return a Python object containing the parsed JSON data as a dictionary or list (depending on what JSON is returned):

In [ ]:
data = resp.json()
data[0]["title"]

Since the results retrieved are based on real-time data, what you see when you run this code will almost definitely be different.

Each element in data is a dictionary containing all of the data found on a GitHub issue page (except for the comments). We can pass data directly to pandas.DataFrame and extract fields of interest:

In [ ]:
issues = pd.DataFrame(data, columns=["number", "title",
                                     "labels", "state"])
issues

With a bit of elbow grease, you can create some higher-level interfaces to common web APIs that return DataFrame objects for more convenient analysis.

## Interacting with Databases
In a business setting, a lot of data may not be stored in text or Excel files. SQL-based relational databases (such as SQL Server, PostgreSQL, and MySQL) are in wide use, and many alternative databases have become quite popular. The choice of database is usually dependent on the performance, data integrity, and scalability needs of an application.

pandas has some functions to simplify loading the results of a SQL query into a DataFrame. As an example, I’ll create a SQLite3 database using Python’s built-in sqlite3 driver:

In [ ]:
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect("mydata.sqlite")
con.execute(query)
con.commit()

Then, insert a few rows of data:



In [ ]:
data = [("Atlanta", "Georgia", 1.25, 6),
        ("Tallahassee", "Florida", 2.6, 3),
        ("Sacramento", "California", 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

Most Python SQL drivers return a list of tuples when selecting data from a table:



In [ ]:
cursor = con.execute("SELECT * FROM test")
rows = cursor.fetchall()
rows

You can pass the list of tuples to the DataFrame constructor, but you also need the column names, contained in the cursor’s description attribute. Note that for SQLite3, the cursor description only provides column names (the other fields, which are part of Python's Database API specification, are None), but for some other database drivers, more column information is provided:

In [ ]:
cursor.description
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

This is quite a bit of munging that you’d rather not repeat each time you query the database. The SQLAlchemy project is a popular Python SQL toolkit that abstracts away many of the common differences between SQL databases. pandas has a read_sql function that enables you to read data easily from a general SQLAlchemy connection. You can install SQLAlchemy with conda like so:

conda install sqlalchemy
Now, we'll connect to the same SQLite database with SQLAlchemy and read data from the table created before:

In [ ]:
import sqlalchemy as sqla
db = sqla.create_engine("sqlite:///mydata.sqlite")
pd.read_sql("SELECT * FROM test", db)

In [ ]:
!rm mydata.sqlite